### Non-exhaustive test of E6C calculations: verify orientation, U, UB, and rotation directions

#### with the aid of Yong Chu's mental math
#### the TL;DR is that it appears to function as documented and as expected

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gi
gi.require_version('Hkl', '5.0')

from hkl.calc import CalcE6C
from hkl.util import Lattice

### Initialize the calculation engine

In [2]:
e6c_calc = CalcE6C(engine='hkl')
e6c_calc.engine.mode = 'constant_chi_vertical'
e6c_calc.wavelength = 1.  # angstrom

### Setup the crystal lattice

In [3]:
lattice = Lattice(a=1, b=1, c=1, alpha=90, beta=90, gamma=90)
sample = e6c_calc.new_sample('sample0', lattice=lattice)

print('lattice', sample.lattice)
print('physical axes', e6c_calc.physical_axes)
print('pseudo axes', e6c_calc.pseudo_axes)
print('omega parameter is', e6c_calc['omega'])

lattice LatticeTuple(a=1.0, b=1.0, c=1.0, alpha=90.0, beta=90.0, gamma=90.0)
physical axes OrderedDict([('mu', 0.0), ('omega', 0.0), ('chi', 0.0), ('phi', 0.0), ('gamma', 0.0), ('delta', 0.0)])
pseudo axes OrderedDict([('h', 0.0), ('k', 0.0), ('l', 0.0)])
omega parameter is CalcParameter(name='omega', limits=(-180.0, 180.0), value=0.0, fit=True, inverted=False, units='Degree')


### Compute the UB matrix from two reflections

In [4]:
# checking orientation of delta
r1p = e6c_calc.Position(mu=0.0, omega=30.0, chi=0.0, phi=0.0, gamma=0., delta=60.)
r1 = sample.add_reflection(0, 0, 1, position=r1p)
r2p = e6c_calc.Position(mu=0.0, omega=120.0, chi=0.0, phi=0.0, gamma=0, delta=60.)
r2 = sample.add_reflection(1, 0, 0, position=r2p)
sample.compute_UB(r1, r2)

1

In [5]:
sample.U

array([[ 1.00000000e+00, -3.74939946e-33,  6.12323400e-17],
       [ 0.00000000e+00,  1.00000000e+00,  6.12323400e-17],
       [-6.12323400e-17, -6.12323400e-17,  1.00000000e+00]])

In [6]:
sample.UB

array([[ 6.28318531e+00, -3.84734139e-16,  0.00000000e+00],
       [ 0.00000000e+00,  6.28318531e+00,  0.00000000e+00],
       [-3.84734139e-16, -3.84734139e-16,  6.28318531e+00]])

### pause to contemplate life and calculate some motor positions

In [7]:
e6c_calc.physical_positions = e6c_calc.Position(mu=0.0, omega=30.0, chi=90.0, phi=0.0, gamma=0, delta=60.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (0,1,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )


pseudo should be (0,1,0) = (0.000000 1.000000 0.000000)


In [8]:
# checking orientation of delta
e6c_calc.physical_positions = e6c_calc.Position(mu=30.0, omega=0.0, chi=0.0, phi=0.0, gamma=60., delta=0.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (0,1,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )

pseudo should be (0,1,0) = (0.000000 1.000000 0.000000)


In [9]:
e6c_calc.physical_positions = e6c_calc.Position(mu=0, omega=30., chi=-90.0, phi=0.0, gamma=0., delta=60.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (0,-1,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )


pseudo should be (0,-1,0) = (0.000000 -1.000000 0.000000)


In [10]:

e6c_calc.physical_positions = e6c_calc.Position(mu=0.0, omega=-60.0, chi=0.0, phi=0.0, gamma=0, delta=60.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (-1,0,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )


pseudo should be (-1,0,0) = (-1.000000 0.000000 0.000000)


### Diffracting upside-down now
#### Note that omega and phi only need to sum to +-120, which reflects what the inverse calculations from the library give

In [11]:
e6c_calc.physical_positions = e6c_calc.Position(mu=0.0, omega=-50.0, chi=0.0, phi=-70.0, gamma=0, delta=-60.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (1,0,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )

e6c_calc.physical_positions = e6c_calc.Position(mu=0.0, omega=-100.0, chi=0.0, phi=-20.0, gamma=0, delta=-60.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (1,0,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )

e6c_calc.physical_positions = e6c_calc.Position(mu=0.0, omega=100.0, chi=0.0, phi=-220.0, gamma=0, delta=-60.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (1,0,0) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )

pseudo should be (1,0,0) = (1.000000 0.000000 0.000000)
pseudo should be (1,0,0) = (1.000000 0.000000 0.000000)
pseudo should be (1,0,0) = (1.000000 0.000000 0.000000)


In [12]:
e6c_calc.physical_positions = e6c_calc.Position(mu=0.0, omega=45.0, chi=45.0, phi=0.0, gamma=0, delta=90.)
pa = e6c_calc.pseudo_axes
print(f'pseudo should be (0,1,1) = ({pa["h"]:f} {pa["k"]:f} {pa["l"]:f})', )

pseudo should be (0,1,1) = (0.000000 1.000000 1.000000)


In [13]:
for sol in e6c_calc.forward((1,0,0)):
    print(sol.omega + sol.phi, "expected:", abs(abs(sol.omega + sol.phi)-120) < 1e-4)

119.9999999269113 expected: True
-119.9999999269113 expected: True
